In [1]:
# import environment building stuff from gym
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

# import additional helping libraries
import numpy as np
import pandas as pd
import random
import os
from random import randint

# import stable baelines RL algorithms
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
# Class for Job
#-------------------------------------------------------------------------------
class Job:
    def __init__(self, job_id, time_proc, time_rel, time_due, job_core):
        self.job_id = job_id
        self.time_proc = time_proc
        self.time_rel = time_rel
        self.time_due = time_due
        self.job_core = job_core
    
    def func_print(self):
        print(self.job_id)

# Class for Machines
#--------------------------------------------------------------------------------

class Machine:
    def __init__(self, machine_id):
        self.machine_id = machine_id
        self.accepted_jobs = 0
        self.current_load = 0
        self.remaining_load = 0

        self.keys = ['job_id','time_rel','time_proc','time_due','time_start','time_comp']
        self.job_on_machine = dict((k, []) for k in self.keys)
    
    def data_entry(self,job_id,time_rel,time_proc,time_due,time_start,time_comp):
        self.job_on_machine['job_id'].append(job_id)
        self.job_on_machine['time_rel'].append(time_rel)
        self.job_on_machine['time_proc'].append(time_proc)
        self.job_on_machine['time_due'].append(time_due)
        self.job_on_machine['time_start'].append(time_start)
        self.job_on_machine['time_comp'].append(time_comp)

In [3]:
# function to create machine list
def initialize_machines(no_of_machines):
    list_machines = []
    for m in range (0, no_of_machines):
        machine_id = "M" + str(m+1)  # Assigning unique id to each machine
        list_machines.append(Machine(machine_id))
    return list_machines

In [4]:
# Function for reading jobs from the text file
#---------------------------------------------------------------------------------

def read_jobs(file_name,no_of_jobs):
    
    file_name = os.path.join("Test_Data",file_name)
    with open(file_name) as read_file:
        file_data = read_file.readlines()
    read_file.close()

    jobs = []
    if no_of_jobs == ':':
        for line in file_data[:]:
            job_properties = line.split(";")
            job_id = job_properties[0]
            time_proc = int(job_properties[1])
            time_rel = int(job_properties[2])
            time_due = int(job_properties[3])
            job_core = int(job_properties[4])
            j = Job(job_id, time_proc, time_rel, time_due, job_core)
            jobs.append(j)
    else:
        for line in file_data[:no_of_jobs]:
            job_properties = line.split(";")
            job_id = job_properties[0]
            time_proc = int(job_properties[1])
            time_rel = int(job_properties[2])
            time_due = int(job_properties[3])
            job_core = int(job_properties[4])
            j = Job(job_id, time_proc, time_rel, time_due, job_core)
            jobs.append(j)

    return jobs

In [5]:
# Function to calculate remaining load on the machines at present time
def update_all_machines(list_machines,job):
    for j in range(0,len(list_machines)):
        remaining_load =   list_machines[j].current_load - job.time_rel
        remaining_load  =  remaining_load if remaining_load >=0 else 0
        list_machines[j].remaining_load = remaining_load
    
    list_machines.sort(key=lambda Machine: Machine.remaining_load,reverse=True)
    return list_machines

In [6]:
# Function to assign jobs to machines
def assign_job_to_machine(action,list_machines, job):
    
    job_assigned = False
    if not action:
        return list_machines,job_assigned
    for j in range(0,len(list_machines)):                    
        # Checking for legality and assigning with Greedy Best Fit method
        if job.time_due >= job.time_proc + job.time_rel + list_machines[j].remaining_load:

            list_machines[j].current_load = job.time_proc +job.time_rel + list_machines[j].remaining_load    
            list_machines[j].accepted_jobs += job.time_proc
            list_machines[j].remaining_load += job.time_proc
            job_assigned = True

            break
    list_machines.sort(key=lambda Machine: Machine.remaining_load,reverse=True)
    return list_machines,job_assigned

In [7]:
# Function to retrieve the observation space (remaining load on the machines) 
def remaining_load_on_machines(list_machines):
    remaining_loads_list = []
    for j in range(0,len(list_machines)):
        remaining_loads_list.append(list_machines[j].remaining_load)
    return remaining_loads_list

In [8]:
def binarySearchMean(list_machines, l, r, x):
  
    # Check base case
    if r >= l:
  
        mid = l + (r - l) // 2
            
        # If element is present at the middle itself
        if list_machines[mid].remaining_load < x and list_machines[mid-1].remaining_load>= x:
            return mid

        # If element is smaller than mid, then it
        # can only be present in left subarray
        elif list_machines[mid].remaining_load < x:
            return binarySearchMean(list_machines, l, mid-1, x)

        # Else the element can only be present
        # in right subarray
        else:
            return binarySearchMean(list_machines, mid + 1, r, x)

            
    else:
        # Element is not present in the array
        return -1

In [9]:
def leap_calc(list_machines,l,r,remaining_load_prime):
        # Check base case
    if r >= l:
  
        mid = l + (r - l) // 2
  
        # If element is present at the middle itself
        if list_machines[mid].remaining_load < remaining_load_prime and list_machines[mid-1].remaining_load> remaining_load_prime:
            return mid
  
        # If element is smaller than mid, then it
        # can only be present in left subarray
        elif list_machines[mid].remaining_load < remaining_load_prime:
            return leap_calc(list_machines, l, mid-1, remaining_load_prime)
  
        # Else the element can only be present
        # in right subarray
        else:
            return leap_calc(list_machines, mid + 1, r, remaining_load_prime)
  
    else:
        # Element is not present in the array
        return -1 
        

In [10]:
def profitability_check(list_machines,job):
    
    # Find the mean of the remaining load on the machines
    mean_load = sum([j.remaining_load for j in list_machines])/len(list_machines)
    pos_mean = binarySearchMean(list_machines,0,len(list_machines)-1,mean_load)
    threshold = len(list_machines) - pos_mean
    leap_pos = leap_calc(list_machines,0,len(list_machines)-2,list_machines[-1].remaining_load + job.time_proc)
    
    if leap_pos <= threshold:
        profitable = True
    else :
        profitable = False
    
    return profitable

In [11]:
def box_plot_data(list_machines):
    data = remaining_load_on_machines(list_machines)
    median = np.median(data)
    upper_quartile = np.percentile(data, 75)
    lower_quartile = np.percentile(data, 25)
    max_value = data[0]
    min_value = data[-1]
    
    return [max_value, upper_quartile, median, lower_quartile, min_value]
    


In [13]:
# Custom environment for scheduling process
class ProcessEnv(Env):
    
    """
        ### Methodology
        -----------------------------------------
        For every incoming job the profitability is checked and then the job is passed to the assign_job_to_machine()
        function which returns job_assigned(True/False).
        The rewards for learning are given as specified under Rewards section.
        The learning algorithm tries to maximize profitable actions.
        
        ### Action Space
        -----------------------------------------
        The agent take a 1-element vector for actions.
        The action space is `(action)` in `[0, 1]`, where `action` is used to accept or reject
        the incoming job:
        
        | Num | Action                 |
        |-----|------------------------|
        | 0   | Reject the job         |
        | 1   | Accept the job         |
        
        ### Observation Space
        -----------------------------------------
        The observation space includes the remaining load summary(box plot 5 point summary) on the machines, the incoming
        job profile(release date, processing time , due date) and the profitability for the incoming job
        
        The observation is a `ndarray` with shape `(9,)` where the elements correspond to the following:
        
        | Num | Observation           | Min                  | Max                |
        |-----|-----------------------|----------------------|--------------------|
        | 0   | max remaining load    | -Inf                 | Inf                |
        | 1   | upper_quartile        | -Inf                 | Inf                |
        | 2   | median                | -Inf                 | Inf                |
        | 3   | lower_quartile        | -Inf                 | Inf                |
        | 4   | min remaining load    | -Inf                 | Inf                |
        | 5   | processing time       | -Inf                 | Inf                |
        | 6   | release time          | -Inf                 | Inf                |
        | 7   | due time              | -Inf                 | Inf                |
        
        ### Rewards
        -----------------------------------------
        The reward scheme works as stated below
        
        | job_assigned | profitable | reward |
        |--------------|------------|--------|
        | True         | True       | +1     |
        | True         | False      | -1     |
        | False        | True       | -1     |
        | False        | False      | +1     |
        
    """
    
    def __init__(self):
        
        self.action_space = Discrete(2)
        
        # machine data
        self.no_of_machines = randint(10, 200)
        #self.no_of_machines = 150

        self.observation_space = Box(low=-np.inf, high=np.inf,shape=(5+3,), dtype=np.float32)
        
        
        # Set no of process
        self.no_of_jobs = ':'      
        self.rejected_jobs = 0
        
        # Set no of machines
        self.list_machines = initialize_machines(self.no_of_machines)
        #self.remaining_load_threshold  = 0.5
       
        # Set start state of machines
        self.state = 0 
        self.jobs = read_jobs('GoogleTraceD12L60M0.1SD0.04C1S1.txt',self.no_of_jobs)
        self.job = self.jobs[0]
        self.reward = 0
        self.iter_value = 0
        self.prev_reward = 0
        
        self.total_proc_time = 0
        self.accepted_jobs = 0
        
    def step(self, action):
        done = False
        info = { }

    
        if self.iter_value < len(self.jobs): 
            
            # get the next job from the jobs list
            self.job = self.jobs[self.iter_value]
            # update the current remaining load on all the machines
            self.list_machines = update_all_machines(self.list_machines,self.job)
            # check the profitability of accepting this job
            profitable = profitability_check(self.list_machines,self.job)
            # take the action specified by the learning model 
            self.list_machines,job_assigned = assign_job_to_machine(action,self.list_machines, self.job)     
            # update the observation space
            obs_list = box_plot_data(self.list_machines)
            obs_list.append(self.job.time_proc)
            obs_list.append(self.job.time_rel)
            obs_list.append(self.job.time_due)
            #obs_list.append(profitable)
            
            # assign rewards 
            if job_assigned and profitable :
                #self.total_proc_time += self.job.time_proc
                self.reward += 1
                #self.accepted_jobs += 1
            elif job_assigned and not profitable:
                #self.total_proc_time += self.job.time_proc
                self.reward -= 1
                #self.accepted_jobs += 1
            elif not job_assigned and profitable:
                self.reward -= 1
            else:
                self.reward += 1        
            
            self.iter_value += 1
        else:
            self.reward = self.reward - self.prev_reward
            self.prev_reward  = self.reward
            obs_list = [0]*(5+3)
            #print('Proc_time:{} Accepted_jobs:{}'.format(self.total_proc_time,self.accepted_jobs))
            #print('Prev_Reward:{}'.format(self.prev_reward))
            done = True
        
        # Return step information
        return np.array(obs_list), self.reward, done,info

    def render(self):
        pass
    
    def reset(self):
        # Reset machines
        self.no_of_machines = randint(10, 200)
        #self.no_of_machines = 150
        
        # Set no of process
        self.no_of_jobs = ':'      
        self.rejected_jobs = 0
        
        # Set no of machines
        self.list_machines = initialize_machines(self.no_of_machines)
        
        # Set start state of machines
        self.state = 0 
        self.jobs = read_jobs('GoogleTraceD12L60M0.1SD0.04C1S1.txt',env.no_of_jobs)
        self.job = self.jobs[0]
        self.reward = 0
        self.iter_value = 0
        self.prev_reward = 0
        
        self.total_proc_time = 0
        self.accepted_jobs = 0
        
        return np.array([0]*(5+3)).astype(float)
    
    
env=ProcessEnv()
from stable_baselines3.common.env_checker import check_env
check_env(env, warn=True)

In [14]:
# loaction of model
models_dir = "models/DQN"

# defining lists to keep track of info
machines_used = []
accepted_load_average = []
rejected_load_average = []
available_resource_list = []
# reading all the files from folder Test_Data
files_list = []
for dirName, subDirList, fileList in os.walk('Test_Data'):
    for file in fileList: 
        files_list.append(file)
        
# starting from first file
current_file_count = 0

# traverse through all the files
while current_file_count < len(files_list):
    
    # repeat the same set of 10 files for each no of machines used
    current_file_count_before_loop = current_file_count
    # traverse through the same set of 10 files for machines from 10 to 190
    for no_machines in range(10,200,40):
        
        # repeat the same set of 10 files for each no of machines used
        current_file_count = current_file_count_before_loop
        
        # initializations for average calc
        total_load_accepted_sum = 0
        total_load_rejected_sum = 0
        available_resource_sum = 0
        
        while current_file_count < (current_file_count_before_loop + 10):
            
            # reset the learning environment
            env = ProcessEnv()  
            env.reset()
            
            # initialize the machine class
            list_machines = initialize_machines(no_machines)  
            total_accepted = 0
            total_rejected = 0
            
            # read data from file
            jobs = read_jobs(files_list[current_file_count],':')

            # set the model to be used 
            model_path = f"{models_dir}/14000000.zip"
            model = DQN.load(model_path, env=env)

            # iterate through the file and schedule jobs as per model prediction
            for job in jobs:
                list_machines = update_all_machines(list_machines,job)

                # update the observation space
                obs_list = box_plot_data(list_machines)
                obs_list.append(job.time_proc)
                obs_list.append(job.time_rel)
                obs_list.append(job.time_due)

                action, _states = model.predict(np.array(obs_list))
                list_machines,job_assigned = assign_job_to_machine(action,list_machines, job)

                if job_assigned: 
                    total_accepted += job.time_proc
                elif not job_assigned:
                    total_rejected += job.time_proc
            # to know the highest completion time of the jobs assigned to machines
            list_machines.sort(key=lambda Machine: Machine.current_load,reverse=True)
            
            C_max = list_machines[0].current_load*no_machines
            total_submitted_load = total_accepted + total_rejected
            available_resource = min(C_max,total_submitted_load)
            
            available_resource_sum += available_resource
            total_load_accepted_sum += total_accepted
            total_load_rejected_sum += total_rejected 
            
            # go to next file
            current_file_count += 1
        
        accepted_load_average.append(total_load_accepted_sum/10)
        rejected_load_average.append(total_load_rejected_sum/10)
        available_resource_list.append(available_resource_sum/10)
        machines_used.append(no_machines)
        
    print('file no: {}'.format(current_file_count))

# store the infos to dictionary and extract to Results.csv        
dict = {'Machines':machines_used,'Accepted Load':accepted_load_average,'Rejected Load':rejected_load_average,'Available Resource':available_resource_list}    
df = pd.DataFrame(dict) 
df.to_csv('Results_D12.csv')

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

...

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
file no: 900
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
file no: 920
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.